In [1]:
import os
import requests
from tqdm import tqdm

URL = "https://download.openmmlab.com/mmaction/v1.0/skeleton/data/ucf101_2d.pkl"
OUT_DIR = "./../data"
OUT_FILE = os.path.join(OUT_DIR, "ucf101_2d.pkl")

os.makedirs(OUT_DIR, exist_ok=True)

print("Download target:", OUT_FILE)

Download target: ./../data/ucf101_2d.pkl


In [5]:
def get_remote_filesize(url):
    try:
        r = requests.head(url, allow_redirects=True, timeout=10)
        file_size = int(r.headers.get("content-length", 0))
        return file_size
    except:
        return None

1070780736

In [6]:
remote_size = get_remote_filesize(URL)
remote_size

1070780736

In [15]:
def download_file(url, out_path, chunk_size=1024*1024,):
    temp_size = 0
    if os.path.exists(out_path):
        temp_size = os.path.getsize(out_path)

    print(f"Local existing size: {temp_size} bytes")
    print(f"Remote total size:   {remote_size} bytes")

    if remote_size is not None and temp_size == remote_size:
        print("File already fully downloaded!")
        return

    headers = {"Range": f"bytes={temp_size}-"} if temp_size > 0 else None

    with requests.get(url, stream=True, headers=headers, timeout=20) as r:
        r.raise_for_status()
        total_to_download = remote_size - temp_size if remote_size else None

        with open(out_path, "ab") as f:
            pbar = tqdm(
                total=total_to_download,
                unit="B",
                unit_scale=True,
                desc="Downloading",
                initial=temp_size,
            )

            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

            pbar.close()


download_file(URL, OUT_FILE)

Local existing size: 0 bytes
Remote total size:   1070780736 bytes


Downloading: 100%|██████████| 1.07G/1.07G [03:11<00:00, 5.60MB/s]


In [16]:
local_size = os.path.getsize(OUT_FILE)
print("Local file size:", local_size)
print("Remote file size:", remote_size)

if remote_size is not None and local_size == remote_size:
    print("SUCCESS: Download completed and verified!")
else:
    print("WARNING: File size mismatch or remote size unavailable.")

Local file size: 1070780736
Remote file size: 1070780736
SUCCESS: Download completed and verified!


In [18]:
import pickle

try:
    with open(OUT_FILE, "rb") as f:
        data = pickle.load(f)
    print("Pickle loaded successfully!")
    print("Top-level keys:", data.keys())
    print("Number of annotations:", len(data["annotations"]))
    print("Splits:", {k: len(v) for k,v in data["split"].items()})
except Exception as e:
    print("Error loading pickle:", e)

Pickle loaded successfully!
Top-level keys: dict_keys(['split', 'annotations'])
Number of annotations: 13320
Splits: {'train1': 9537, 'train2': 9586, 'train3': 9624, 'test1': 3783, 'test2': 3734, 'test3': 3696}
